# Publishing ModelTrainer Models to SageMaker Endpoints

## Finding the Latest ModelTrainer Job


In [ ]:
import sagemaker

# Create a SageMaker session
sagemaker_session = sagemaker.Session()

# List most recent completed training job
training_jobs = sagemaker_session.sagemaker_client.list_training_jobs(
    SortBy='CreationTime',                 # Sort jobs by creation time
    SortOrder='Descending',                # Newest jobs first
    StatusEquals='Completed',              # Only include completed jobs
    NameContains='sklearn-modeltrainer'    # Filter to ModelTrainer jobs
)

# Extract the name of the latest training job
TRAINING_JOB_NAME = training_jobs['TrainingJobSummaries'][0]['TrainingJobName']

## Retrieving Model Artifacts from ModelTrainer Jobs


In [ ]:
# Get training job details
training_job_details = sagemaker_session.describe_training_job(TRAINING_JOB_NAME)

# Extract S3 location of model artifacts
model_data = training_job_details['ModelArtifacts']['S3ModelArtifacts']

print("Model Artifacts URI:", model_data)


## Entry Point Script for Inference

Created in the entry_point.py file

## Creating the Model Object


In [ ]:
from sagemaker.sklearn.model import SKLearnModel

# Get account id and role
account_id = sagemaker_session.boto_session.client('sts').get_caller_identity()['Account']
SAGEMAKER_ROLE = f"arn:aws:iam::{account_id}:role/SageMakerDefaultExecution"

# Create SKLearnModel using ModelTrainer artifacts
model = SKLearnModel(
    model_data=model_data,               # S3 location from ModelTrainer job
    role=SAGEMAKER_ROLE,                 # IAM role for SageMaker
    entry_point='entry_point.py',        # Inference script
    framework_version='1.2-1',           # scikit-learn version
    py_version='py3',                    # Python version
    sagemaker_session=sagemaker_session  # SageMaker session
)

## Deploying to the Serverless Endpoint


In [ ]:
from sagemaker.serverless import ServerlessInferenceConfig

# Unique name for the SageMaker endpoint to be created
ENDPOINT_NAME = "california-housing-modeltrainer"

# Configure serverless inference (same as previous lessons)
serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=2048,
    max_concurrency=10
)

# Deploy as serverless endpoint asynchronously
predictor = model.deploy(
    serverless_inference_config=serverless_config,
    endpoint_name=ENDPOINT_NAME,
    wait=False
)

# Retrieve detailed information about the specified SageMaker endpoint
endpoint_description = sagemaker_session.sagemaker_client.describe_endpoint(EndpointName=ENDPOINT_NAME)

# Extract the current status of the endpoint from the response
status = endpoint_description['EndpointStatus']

# Display the endpoint's status
print(f"Endpoint status: {status}")